In [1]:
import pandas as pd
import seaborn as sns

from pyDatalog import pyDatalog
from pyDatalog.pyDatalog import assert_fact, load, ask

In [2]:
path = 'AMIERules/'

In [3]:
result = pd.read_csv(path + 'FrenchRoyalty_AMIE_Rules.csv') #, delimiter='\t'
result.Body = result.Body.str.rstrip()
result.Body
display(result.head(), result.shape)

,Body,Head,Head Coverage,Std Confidence,PCA Confidence,Positive Examples,Body size,PCA Body size,Functional variable
0,?b <predecessor> ?a,?a <successor> ?b,1.000000,1.0,1.0,471,471,471,?a
1,?b <successor> ?a,?a <predecessor> ?b,1.000000,1.0,1.0,471,471,471,?b
2,?a <hasSpouse> <Yes>,?a <type> <Person>,0.447345,1.0,1.0,994,994,994,?a
3,"?a <gender> ""male""@en",?a <type> <Person>,0.260126,1.0,1.0,578,578,578,?a
4,"?a <gender> ""female""@en",?a <type> <Person>,0.211971,1.0,1.0,471,471,471,?a


(794, 9)

## Intensional Data Base

In [4]:
amie_rule = result[['Body', 'Head']]
terms = set()

for d in amie_rule.values:
    rule = """"""
    
#     === Creating Head of the Rule ===
    head = d[1].split('  ')
    sub, pred, obj = head[0], head[1], head[2]
    terms.add(pred)
    sub = sub.replace('?', '')
    sub = sub.capitalize()
    terms.update(sub)
    rule+=pred+'('+sub+', '
    
    if obj.startswith('?'):
            obj = obj.replace('?', '')
            obj = obj.capitalize()
            terms.update(obj)
            rule+=obj+') <= '
    else:
        rule+="""'"""+obj+"""') <= """
    
#     === Creating Body of Rule ===
    literal = d[0].split('  ')
    for i in range(2, len(literal), 3):
        sub, pred, obj = literal[i-2], literal[i-1], literal[i]
        terms.add(pred)
        if sub.startswith('?'):
            sub = sub.replace('?', '')
            sub = sub.capitalize()
            terms.update(sub)
            rule+=pred+'('+sub+', '
        else:
            rule+=pred+"""('"""+sub+"""', """
        if obj.startswith('?'):
            obj = obj.replace('?', '')
            obj = obj.capitalize()
            terms.update(obj)
            rule+=obj+') & '
        else:
            rule+="""'"""+obj+"""') & """
    rule = rule[:-3]
        
    print(rule)   

<successor>(A, B) <= <predecessor>(B, A)
<predecessor>(A, B) <= <successor>(B, A)
<type>(A, '<Person>') <= <hasSpouse>(A, '<Yes>')
<type>(A, '<Person>') <= <gender>(A, '"male"@en')
<type>(A, '<Person>') <= <gender>(A, '"female"@en')
<type>(A, '<Person>') <= <hasSpouse>(A, '<No>')
<parent>(A, B) <= <father>(A, B)
<parent>(A, B) <= <mother>(A, B)
<gender>(A, '"male"@en') <= <father>(D, A) & <successor>(A, D)
<gender>(A, '"male"@en') <= <child>(A, D) & <father>(D, A)
<gender>(A, '"male"@en') <= <father>(C, A) & <parent>(C, A)
<gender>(A, '"male"@en') <= <father>(C, A) & <predecessor>(C, A)
<gender>(A, '"male"@en') <= <father>(C, A) & <gender>(C, '"female"@en')
<gender>(A, '"male"@en') <= <father>(C, A) & <type>(C, '<Person>')
<gender>(A, '"male"@en') <= <father>(C, A) & <hasSpouse>(C, '<No>')
<gender>(A, '"male"@en') <= <father>(C, A) & <hasSpouse>(C, '<Yes>')
<gender>(A, '"female"@en') <= <mother>(D, A) & <successor>(A, D)
<gender>(A, '"female"@en') <= <child>(A, D) & <mother>(D, A)
<gen

## Extensional Data base

In [5]:
path = '../KG/FrenchRoyalty/'

In [6]:
data = pd.read_csv(path + 'french_training.tsv', delimiter='\t', header=None) #
data.columns = ['s', 'p', 'o']
data.head()

,s,p,o
0,<1383_1385_Portuguese_interregnum>,<hasSpouse>,<No>
1,<1383_1385_Portuguese_interregnum>,<successor>,<John_I_of_Portugal>
2,<1383_1385_Portuguese_interregnum>,<type>,<Person>
3,<A_nor_de_Ch_tellerault>,<father>,<Aimery_I_Viscount_of_Ch_tellerault>
4,<A_nor_de_Ch_tellerault>,<gender>,"""female""@en ."


In [7]:
terms.update(set(data.p.unique()))
terms = ','.join(terms)
terms

'D,<child>,<father>,<predecessor>,<parent>,A,<gender>,F,<name>,E,C,<successor>,<type>,<marriedTo>,B,<hasSpouse>,<mother>,<spouse>'

In [8]:
pyDatalog.create_terms(terms)

def build_datalog_model(data):
    pyDatalog.clear()
    for d in data.values:
        # Extensional Database
        assert_fact(d[1], d[0], d[2])
        
#         === TO DO.... ===
#     + gender(A, '<female>') <= parent(A, '""@jfg.') & successor(D, A)